https://udlbook.github.io/udlbook/

https://github.com/udlbook/udlbook/blob/main/Notebooks/Chap07/7_2_Backpropagation.ipynb

# **Блокнот 7.2: Обратное распространение (backpropagation)**

В этом блокноте выполняется алгоритм обратного распространения ошибки в глубокой нейронной сети, как описано в разделе 7.4 книги.

Пройдитесь по ячейкам ниже, запуская каждую ячейку по очереди. В разных местах вы увидите метку "TODO". Следуйте инструкциям в этих местах и сделайте прогнозы о том, что должно произойти, или напишите код для выполнения функций.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

Сначала давайте определим нейронную сеть. Пока мы просто выберем веса и смещения случайным образом

In [5]:
# Установим начальное значение (seed), чтобы мы всегда получали одни и те же случайные числа
np.random.seed(0)

# Количество слоев
K = 5
# Количество нейронов на слой
D = 6
# Входной слой
D_i = 1
# Выходной слой
D_o = 1

# Сделаем пустые списки
all_weights = [None] * (K+1)
all_biases = [None] * (K+1)

# Создадим входной и выходной слои
all_weights[0] = np.random.normal(size=(D, D_i))
all_weights[-1] = np.random.normal(size=(D_o, D))
all_biases[0] = np.random.normal(size =(D,1))
all_biases[-1]= np.random.normal(size =(D_o,1))

# Создадим промежуточные слои
for layer in range(1,K):
  all_weights[layer] = np.random.normal(size=(D,D))
  all_biases[layer] = np.random.normal(size=(D,1))

In [6]:
# Определим функцию активации ReLU (Rectified Linear Unit)
def ReLU(preactivation):
  activation = preactivation.clip(0.0)
  return activation

Теперь давайте запустим нашу случайную сеть.  Матрицы весов $\boldsymbol\Omega_{1\ldots K}$ являются элементами списка "all_weights", а смещения $\boldsymbol\beta_{1\ldots k}$ являются элементами списка "all_biases"

Мы знаем, что нам понадобятся предварительные активации $\mathbf{f}_{0\ldots K}$ и активации $\mathbf{h}_{1\ldots K}$ для прямого прохода, поэтому мы также сохраним и вернем их.


In [7]:
def compute_network_output(net_input, all_weights, all_biases):

  # Получим количество слоев
  K = len(all_weights) -1

  # Сохраним предварительные активации на каждом слое в список "all_f",
  # а активации - во второй список "all_h".
  all_f = [None] * (K+1)
  all_h = [None] * (K+1)


  # Для удобства мы установим
  # all_h[0] в качестве входа, а all_f[K] будет выходом
  all_h[0] = net_input

  # Пройдемся по слоям, вычисляя all_f[0...K-1] и all_h[1...K]
  for layer in range(K):
      # Обновите преактивации и активации на этом слое в соответствии с уравнением 7.16
      # Не забудьте использовать np.matmul для умножения матриц
      # TODO - Замените строки ниже
      all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer], all_h[layer])
      all_h[layer+1] = ReLU(all_f[layer])



  # Вычислите выход последнего скрытого слоя
  # TO DO -- Замените строку ниже
  all_f[K] = all_biases[K] + np.matmul(all_weights[K], all_h[K])

  # Извлечем выход
  net_output = all_f[K]

  return net_output, all_f, all_h

In [8]:
# Зададим входные данные
net_input = np.ones((D_i,1)) * 1.2
# Рассчитаем выход нейронной сети
net_output, all_f, all_h = compute_network_output(net_input,all_weights, all_biases)
print("True output = %3.3f, Your answer = %3.3f"%(1.907, net_output[0,0]))

True output = 1.907, Your answer = 1.907


Теперь давайте зададим функцию потерь. Мы будем использовать квадратичную функцию потерь. Мы также напишем функцию для вычисления d_loss_d_output

In [9]:
def least_squares_loss(net_output, y):
  return np.sum((net_output-y) * (net_output-y))

def d_loss_d_output(net_output, y):
    return 2*(net_output-y);

In [10]:
y = np.ones((D_o,1)) * 20.0
loss = least_squares_loss(net_output, y)
print("y = %3.3f Loss = %3.3f"%(y, loss))

y = 20.000 Loss = 327.371


Теперь давайте вычислим производные сети. Мы уже вычислили прямой проход. Давайте вычислим обратный проход.

In [11]:
# Нам понадобится индикаторная функция
def indicator_function(x):
  x_in = np.array(x)
  x_in[x_in>=0] = 1
  x_in[x_in<0] = 0
  return x_in

# Основной расчет обратного прохода
def backward_pass(all_weights, all_biases, all_f, all_h, y):
  # Мы также сохраним производные dl_dweights и dl_dbiases в списках
  all_dl_dweights = [None] * (K+1)
  all_dl_dbiases = [None] * (K+1)
  # И сохраним производные от функции потерь относительно активации и предварительной активации в списках
  all_dl_df = [None] * (K+1)
  all_dl_dh = [None] * (K+1)

  # Опять же, для удобства будем придерживаться соглашения о том, что all_h[0] - это вход сети, а all_f[k] - выход сети


  # Вычислим производные от функции потерь по отношению к выходу сети
  all_dl_df[K] = np.array(d_loss_d_output(all_f[K],y))


  # Теперь пойдем в обратном направлении по сети
  for layer in range(K,-1,-1):
    # TODO Вычислите производные функции потерь относительно смещений на этом слое из all_dl_df[layer]. (уравнение 7.21)
    # ПРИМЕЧАНИЕ.  Чтобы получить копию матрицы X, используйте Z=np.array(X)
    # ЗАМЕНИТЕ ЭТУ СТРОКУ
    all_dl_dbiases[layer] = np.array(all_dl_df[layer])



    # TODO вычислите производные функции потерь относительно весов в слое из all_dl_df[layer] и all_h[layer] (уравнение 7.22)
    # Не забудьте использовать np.matmul
    # ЗАМЕНИТЕ ЭТУ СТРОКУ
    all_dl_dweights[layer] = np.matmul(np.array(all_dl_df[layer]), all_h[layer].T)


    # TODO: вычислите производные фунции потерь относительно активаций из весов и производных от следующих предварительных активаций (вторая часть последней строки уравнения 7.24)
    # Не забудьте использовать np.matmul
    # ЗАМЕНИТЕ ЭТУ СТРОКУ
    all_dl_dh[layer] = np.matmul(all_weights[layer].T, all_dl_df[layer])

    if layer > 0:
      # TODO Вычислите производные функции потерь относительно предварительной активации f (используйте производную функции ReLU, первую часть последней строки уравнения 7.24)
      # ЗАМЕНИТЕ ЭТУ СТРОКУ
      all_dl_df[layer-1] = indicator_function(all_f[layer-1])*all_dl_dh[layer]

  return all_dl_dweights, all_dl_dbiases

In [12]:
all_dl_dweights, all_dl_dbiases = backward_pass(all_weights, all_biases, all_f, all_h, y)

In [13]:
np.set_printoptions(precision=3)
# Создайте пустые списки для производных, вычисленных с помощью конечных разностей
all_dl_dweights_fd = [None] * (K+1)
all_dl_dbiases_fd = [None] * (K+1)

# Давайте проверим, правильно ли мы вычисляем производные,
# сравнив их со значениями, рассчитанными с помощью конечных разностей
delta_fd = 0.000001

# Проверим производные по векторам смещения
for layer in range(K):
  dl_dbias  = np.zeros_like(all_dl_dbiases[layer])
  # Для каждого элемента в смещении
  for row in range(all_biases[layer].shape[0]):
    # Сделаем копию смещений, мы будем менять по одному элементу каждый раз.
    all_biases_copy = [np.array(x) for x in all_biases]
    all_biases_copy[layer][row] += delta_fd
    network_output_1, *_ = compute_network_output(net_input, all_weights, all_biases_copy)
    network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
    dl_dbias[row] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dbiases_fd[layer] = np.array(dl_dbias)
  print("-----------------------------------------------")
  print("Bias %d, derivatives from backprop:"%(layer))
  print(all_dl_dbiases[layer])
  print("Bias %d, derivatives from finite differences"%(layer))
  print(all_dl_dbiases_fd[layer])
  if np.allclose(all_dl_dbiases_fd[layer],all_dl_dbiases[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")




# Проверим производные по весовым матрицам
for layer in range(K):
  dl_dweight  = np.zeros_like(all_dl_dweights[layer])
  # Для каждого элемента в all_weights
  for row in range(all_weights[layer].shape[0]):
    for col in range(all_weights[layer].shape[1]):
      # Сделаем копию весов, мы будем менять по одному элементу каждый раз.
      all_weights_copy = [np.array(x) for x in all_weights]
      all_weights_copy[layer][row][col] += delta_fd
      network_output_1, *_ = compute_network_output(net_input, all_weights_copy, all_biases)
      network_output_2, *_ = compute_network_output(net_input, all_weights, all_biases)
      dl_dweight[row][col] = (least_squares_loss(network_output_1, y) - least_squares_loss(network_output_2,y))/delta_fd
  all_dl_dweights_fd[layer] = np.array(dl_dweight)
  print("-----------------------------------------------")
  print("Weight %d, derivatives from backprop:"%(layer))
  print(all_dl_dweights[layer])
  print("Weight %d, derivatives from finite differences"%(layer))
  print(all_dl_dweights_fd[layer])
  if np.allclose(all_dl_dweights_fd[layer],all_dl_dweights[layer],rtol=1e-05, atol=1e-08, equal_nan=False):
    print("Success!  Derivatives match.")
  else:
    print("Failure!  Derivatives different.")

-----------------------------------------------
Bias 0, derivatives from backprop:
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Bias 0, derivatives from finite differences
[[ -4.486]
 [  4.947]
 [  6.812]
 [ -3.883]
 [-24.935]
 [  0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 1, derivatives from backprop:
[[ -0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Bias 1, derivatives from finite differences
[[  0.   ]
 [-11.297]
 [  0.   ]
 [  0.   ]
 [-10.722]
 [  0.   ]]
Success!  Derivatives match.
-----------------------------------------------
Bias 2, derivatives from backprop:
[[-0.   ]
 [-0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Bias 2, derivatives from finite differences
[[ 0.   ]
 [ 0.   ]
 [ 0.938]
 [ 0.   ]
 [-9.993]
 [ 0.508]]
Success!  Derivatives match.
-----------------------------------------------
Bias 3, derivatives from backprop:
[[-0.   ]
 [-4.8  ]
 [-1.661]
 [-0.   ]
 [ 3.393]
 [ 5.391]